<a href="https://colab.research.google.com/github/nipun-taneja/amorphous-yolo/blob/main/notebooks/01_baseline_vs_eiou.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Clone your repo into the Colab VM
!git clone https://github.com/nipun-taneja/amorphous-yolo.git
%cd amorphous-yolo
!ls

In [ ]:
!pip install --upgrade pip
!pip install -U "ultralytics" "wandb"


In [ ]:
import wandb
# wandb.login()  # will show a link; paste your API key from wandb.ai

In [ ]:
wandb.login()

In [ ]:
import inspect, ultralytics
from ultralytics.utils import loss as loss_mod

print("Ultralytics version:", ultralytics.__version__)
print(loss_mod.BboxLoss)
print(inspect.getsource(loss_mod.BboxLoss.forward)[:400])


In [ ]:
from src.losses import EIoULoss, AEIoULoss
import torch

print("Ultralytics version:", __import__("ultralytics").__version__)

loss_fn = EIoULoss()

pred = torch.tensor([[0.0, 0.0, 1.0, 1.0],
                     [0.2, 0.2, 0.8, 0.8]])
gt   = torch.tensor([[0.0, 0.0, 1.0, 1.0],
                     [0.0, 0.0, 1.0, 1.0]])

print("EIoU placeholder loss:", loss_fn(pred, gt).item())


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo26n.pt")  # works with 8.4.9 as you saw
results = model.train(
    data="coco8.yaml",          # later replace with your DUO/TrashCan yaml
    epochs=3,
    imgsz=640,
    project="experiments",
    name="baseline_coco8_phase1",
    device=0,
)
print(results)


# Getting data from Roboflow.

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="H4A5U5A1MWhKTjPvmsU1")
project = rf.workspace("amorphousyolo").project("duo-dataset-ofte9")
version = project.version(1)
dataset = version.download("yolo26")


In [ ]:
import random, cv2, matplotlib.pyplot as plt
from pathlib import Path

def show_random(img_root, lbl_root, n=20):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")

        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 5:
                        continue  # skip malformed
                    cls = int(parts[0])
                    x_c, y_c, bw, bh = map(float, parts[1:5])  # ignore extra values

                    x_c *= w; y_c *= h; bw *= w; bh *= h
                    x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
                    x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4, 4))
        plt.title(img_path.name)
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()


In [ ]:
from pathlib import Path

imgs = list(Path("/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/images").glob("*"))
print(len(imgs), "files")
print(imgs[:5])


In [ ]:
show_random(
    "/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/images",
    "/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/labels",
    n=5,
)


In [ ]:
import cv2, matplotlib.pyplot as plt
from pathlib import Path

img_path = Path("/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/images/4828_jpg.rf.9e0a7e8d443c379bf491e4b866b5a524.jpg")
label_path = Path("/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/labels") / (img_path.stem + ".txt")

print("Image exists:", img_path.exists())
print("Label path:", label_path)
print("Label exists:", label_path.exists())
if label_path.exists():
    print("Label contents (first 5 lines):")
    with open(label_path) as f:
        for i, line in enumerate(f):
            print(" ", line.strip())
            if i == 4:
                break

img = cv2.imread(str(img_path))
h, w = img.shape[:2]

if label_path.exists():
    with open(label_path) as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) < 5:
                continue
            cls = int(parts[0])
            x_c, y_c, bw, bh = map(float, parts[1:5])
            x_c *= w; y_c *= h; bw *= w; bh *= h
            x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
            x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

plt.figure(figsize=(4,4))
plt.axis("off")
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()


In [ ]:
import random, cv2, matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

def show_random_poly(img_root, lbl_root, n=5):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")
        print("Image:", img_path.name, "Label exists:", label_path.exists())

        if label_path.exists():
            with open(label_path) as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) < 3:
                        continue
                    cls = int(parts[0])
                    coords = list(map(float, parts[1:]))

                    xs = np.array(coords[0::2]) * w
                    ys = np.array(coords[1::2]) * h

                    x1, x2 = int(xs.min()), int(xs.max())
                    y1, y2 = int(ys.min()), int(ys.max())
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4, 4))
        plt.title(img_path.name)
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

show_random_poly(
    "/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/images",
    "/content/amorphous-yolo/amorphous-yolo/DUO-dataset-1/train/labels",
    n=5,
)


In [ ]:
from ultralytics import YOLO

model = YOLO("yolo26n.pt")
results = model.train(
    data=dataset.location + "/data.yaml",   # Roboflow’s yaml
    epochs=20,
    imgsz=640,
    project="experiments",
    name="baseline_your_dataset",
    device=0,
)


In [ ]:
!mkdir -p "/content/drive/MyDrive/amorphous-yolo"
!cp -r /content/amorphous-yolo/amorphous-yolo/* "/content/drive/MyDrive/amorphous-yolo"

In [ ]:
!mkdir -p "/content/drive/MyDrive/checkpoints/amorphous-yolo"

!cp -r /content/amorphous-yolo/amorphous-yolo/runs/detect/experiments/baseline_your_dataset \
      "/content/drive/MyDrive/checkpoints/amorphous-yolo/"


In [ ]:
import time
for i in range(3600):
  time.sleep(1)


In [ ]:
print("done")

In [ ]:
!pwd

In [ ]:
!cp /content/drive/MyDrive/amorphous-yolo/data/dataset.zip /content/amorphous-yolo/amorphous-yolo/datasets/trashcan.zip

In [ ]:
!mkdir -p /content/datasets/trashcan_raw

In [ ]:
%cd /content/amorphous-yolo/amorphous-yolo/datasets

In [ ]:
!unzip -q /content/amorphous-yolo/amorphous-yolo/datasets/trashcan.zip

In [ ]:
!ls  /content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw


In [ ]:
!tree /content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw

In [ ]:
from pathlib import Path

ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw")
print("instance_version/train files:", len(list((ROOT/"instance_version/train").glob("*"))))
print("instance_version/val files:", len(list((ROOT/"instance_version/val").glob("*"))))
print("original_data/images files:", len(list((ROOT/"original_data/images").glob("*.jpg"))))


In [ ]:
!pip install pycocotools -q

import os
from pathlib import Path
from pycocotools.coco import COCO

ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw")
INSTANCE_DIR = ROOT / "instance_version"
IMAGES_DIR = ROOT / "original_data" / "images"

OUT_ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan")
IMG_TRAIN_DIR = OUT_ROOT / "images" / "train"
IMG_VAL_DIR   = OUT_ROOT / "images" / "val"
LBL_TRAIN_DIR = OUT_ROOT / "labels" / "train"
LBL_VAL_DIR   = OUT_ROOT / "labels" / "val"

for d in [IMG_TRAIN_DIR, IMG_VAL_DIR, LBL_TRAIN_DIR, LBL_VAL_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def coco_to_yolo(json_name, split):
    coco = COCO(str(INSTANCE_DIR / json_name))
    img_id_to_info = coco.imgs
    cat_id_to_idx = {cat_id: i for i, cat_id in enumerate(sorted(coco.cats.keys()))}
    print(f"{split}: {len(img_id_to_info)} images, {len(coco.anns)} anns")

    for img_id, info in img_id_to_info.items():
        file_name = info["file_name"]
        width, height = info["width"], info["height"]

        src_img_path = IMAGES_DIR / file_name
        if split == "train":
            out_img_path = IMG_TRAIN_DIR / file_name
            out_lbl_path = LBL_TRAIN_DIR / (Path(file_name).stem + ".txt")
        else:
            out_img_path = IMG_VAL_DIR / file_name
            out_lbl_path = LBL_VAL_DIR / (Path(file_name).stem + ".txt")

        if not src_img_path.exists():
            continue

        if not out_img_path.exists():
            os.system(f'cp "{src_img_path}" "{out_img_path}"')

        ann_ids = coco.getAnnIds(imgIds=[img_id])
        anns = coco.loadAnns(ann_ids)
        lines = []
        for ann in anns:
            cat_id = ann["category_id"]
            cls = cat_id_to_idx[cat_id]

            x_min, y_min, bw, bh = ann["bbox"]  # COCO bbox
            x_c = (x_min + bw / 2) / width
            y_c = (y_min + bh / 2) / height
            bw_n = bw / width
            bh_n = bh / height

            x_c = min(max(x_c, 0.0), 1.0)
            y_c = min(max(y_c, 0.0), 1.0)
            bw_n = min(max(bw_n, 0.0), 1.0)
            bh_n = min(max(bh_n, 0.0), 1.0)

            lines.append(f"{cls} {x_c:.6f} {y_c:.6f} {bw_n:.6f} {bh_n:.6f}")

        if lines:
            with open(out_lbl_path, "w") as f:
                f.write("\n".join(lines))

coco_to_yolo("instances_train_trashcan.json", "train")
coco_to_yolo("instances_val_trashcan.json", "val")


In [ ]:
from pycocotools.coco import COCO
import yaml
from pathlib import Path

INSTANCE_DIR = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan-raw/instance_version")
OUT_ROOT = Path("/content/amorphous-yolo/amorphous-yolo/datasets/trashcan")

coco_train = COCO(str(INSTANCE_DIR / "instances_train_trashcan.json"))
cats = coco_train.loadCats(coco_train.getCatIds())
cats_sorted = sorted(cats, key=lambda c: c["id"])
names = {i: c["name"] for i, c in enumerate(cats_sorted)}
print("names:", names)

data_dir = Path("/content/amorphous-yolo/amorphous-yolo/data")
data_dir.mkdir(parents=True, exist_ok=True)

yaml_dict = {
    "path": str(OUT_ROOT),
    "train": "images/train",
    "val": "images/val",
    "names": names,
}

with open(data_dir / "trashcan.yaml", "w") as f:
    yaml.safe_dump(yaml_dict, f, sort_keys=False)

print((data_dir / "trashcan.yaml").read_text())


In [ ]:
import random, cv2, matplotlib.pyplot as plt
from pathlib import Path

def show_random_trashcan(img_root, lbl_root, n=10):
    img_paths = list(Path(img_root).glob("*.jpg"))
    random.shuffle(img_paths)

    for img_path in img_paths[:n]:
        img = cv2.imread(str(img_path))
        h, w = img.shape[:2]
        label_path = Path(lbl_root) / (img_path.stem + ".txt")
        print("Image:", img_path.name, "Label exists:", label_path.exists())

        if label_path.exists():
            for line in open(label_path):
                parts = line.strip().split()
                if len(parts) < 5:
                    continue
                _, x_c, y_c, bw, bh = map(float, parts[:5])
                x_c *= w; y_c *= h; bw *= w; bh *= h
                x1, y1 = int(x_c - bw/2), int(y_c - bh/2)
                x2, y2 = int(x_c + bw/2), int(y_c + bh/2)
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)

        plt.figure(figsize=(4,4))
        plt.axis("off")
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        plt.show()

show_random_trashcan(
    "/content/amorphous-yolo/amorphous-yolo/datasets/trashcan/images/train",
    "/content/amorphous-yolo/amorphous-yolo/datasets/trashcan/labels/train",
    n=10,
)


# “Phase 2 – DUO baselines”

In [ ]:
from ultralytics import YOLO

# Start from a small/nano model for speed
model = YOLO("yolo26n.pt")  # or "yolo26s.pt" if you want slightly larger

results_cioU_duo = model.train(
    data="data/duo.yaml",              # Roboflow-copied YAML
    epochs=20,
    imgsz=640,
    project="experiments",
    name="duo_ciou_baseline",
    device=0,
)
